In [ ]:
import numpy as np
a = np.array([40, 45, 55, 50])
b = np.array([30, 45, 25, 50, 40])
c = np.array([[5, 1, 3, 2, 5],
              [4, 4, 3, 4, 7],
              [7, 3, 4, 5, 4],
              [3, 4, 5, 7, 6]])

In [ ]:
def minimum_method(a, b, c):
    local_a = np.copy(a)
    local_b = np.copy(b)
    local_c = np.copy(c)
    local_x = np.zeros(c.shape, dtype=int)

    while(np.sum(local_c) > 0):
        i,j = np.where(local_c==np.min(local_c[np.nonzero(local_c)]))
        i = i[0]
        j = j[0]

        minValue = np.min([local_a[i], local_b[j]])

        local_x[i,j] = minValue

        if(minValue == local_b[j]):
            local_a[i] = local_a[i] - local_b[j]
            local_b[j] = 0
            local_c[:, j] = np.zeros(local_a.shape, dtype=int)
        else:
            local_b[j] = local_b[j] - local_a[i]
            local_a[i] = 0
            local_c[i] = np.zeros(local_b.shape, dtype=int)

    return local_x

In [ ]:
x = minimum_method(a, b, c)
print(x)
print("f =", np.sum(c * x))

[[ 0 40  0  0  0]
 [ 0  0 25 20  0]
 [ 0  5  0 10 40]
 [30  0  0 20  0]]
f = 650


In [ ]:
def fill_potencials(c, is_first_in_c, array_to_fill, array_indexes, array_from_fill, array_from_fill_indexes):
    for i in range(array_from_fill.shape[0]):
        if array_from_fill[i] != 0:
            vdim_indx = np.argwhere(array_from_fill_indexes == i).flatten()
            for id in vdim_indx:
                if(id == 0):
                    continue

                if(array_to_fill[array_indexes[id]] != 0):
                    continue

                if is_first_in_c:
                    array_to_fill[array_indexes[id]] = c[array_indexes[id], array_from_fill_indexes[id]] - array_from_fill[i]
                else:
                    array_to_fill[array_indexes[id]] = c[array_from_fill_indexes[id], array_indexes[id]] - array_from_fill[i]
                array_to_fill[array_indexes[id]] = array_to_fill[array_indexes[id]]  if array_to_fill[array_indexes[id]] > 0 else -1

u = np.zeros(a.shape)
v = np.zeros(b.shape)

u_indexes, v_indexes = np.where(x > 0)

i = 0
while u_indexes[i] == 0:
    x_indx = u_indexes[i]
    y_indx = v_indexes[i]
    v[y_indx] = c[x_indx, y_indx]
    i = i +1

for k in range(20):
    fill_potencials(c, True, u, u_indexes, v, v_indexes)
    fill_potencials(c, False, v, v_indexes, u, u_indexes)

v = np.where(v < 0, 0, v)
u = np.where(u < 0, 0, u)
print("u = ", u, " v = ", v)
print("c matrix")
print(c)
print("solution")
print(x)
x_base = x.copy()

print(np.sum(x * c))

u =  [0. 1. 2. 4.]  v =  [0. 1. 2. 3. 2.]
c matrix
[[5 1 3 2 5]
 [4 4 3 4 7]
 [7 3 4 5 4]
 [3 4 5 7 6]]
solution
[[ 0 40  0  0  0]
 [ 0  0 25 20  0]
 [ 0  5  0 10 40]
 [30  0  0 20  0]]
650


In [ ]:
def get_position(array, x, y):
    corr_x = x
    corr_y = y

    if corr_x >= array.shape[0]:
        corr_x = 0
    elif corr_x < 0:
        corr_x = array.shape[0]-1

    if corr_y >= array.shape[1]:
        corr_y = 0
    elif corr_y < 0:
        corr_y = array.shape[1]-1

    return (corr_x, corr_y)

def check_nodes(node1, node2, node1_modif, node2_modif):

    node3, _ = get_position(x, node1 + node1_modif, node2)
    _, node4= get_position(x, node1, node2 + node2_modif)

    if x[node3, node2] != 0 and x[node1, node4] != 0:
        minValue = np.min([x[node3, node2], x[node1, node4]])

        x[node3, node2] = x[node3, node2] - minValue
        x[node1, node4] = x[node1, node4] - minValue

        x[node1, node2] = x[node1, node2] + minValue
        x[node3, node4] = x[node3, node4] + minValue

In [ ]:
k = 0

while k<1:

    free_cells_x, free_cells_y = np.where(x == 0)

    s = np.zeros(c.shape, dtype=int)

    for i in range(free_cells_x.shape[0]):
        s[free_cells_x[i], free_cells_y[i]] = c[free_cells_x[i], free_cells_y[i]] - u[free_cells_x[i]] - v[free_cells_y[i]]

    negative_s_x, negative_s_y = np.where(s < 0)

    for i in range(negative_s_x.shape[0]):
        negative_s_core_x = negative_s_x[0]
        negative_s_core_y = negative_s_y[0]

        check_nodes(negative_s_core_x, negative_s_core_y, -1, 1)
        check_nodes(negative_s_core_x, negative_s_core_y, 1, -1)
        check_nodes(negative_s_core_x, negative_s_core_y, 1, 1)
        check_nodes(negative_s_core_x, negative_s_core_y, -1, -1)

    print("Iteration", k, "f =", np.sum(c * x))
    k = k + 1

Iteration 0 f = 650


In [ ]:
print("x before method:\n", x_base)
print("x after method:\n", x)
print("s matrix:\n", s)
print("effectivity:", np.sum(c * x))

x before method:
 [[ 0 40  0  0  0]
 [ 0  0 25 20  0]
 [ 0  5  0 10 40]
 [30  0  0 20  0]]
x after method:
 [[ 0 40  0  0  0]
 [ 0  0 25 20  0]
 [ 0  5  0 10 40]
 [30  0  0 20  0]]
s matrix:
 [[ 5  0  1 -1  3]
 [ 3  2  0  0  4]
 [ 5  0  0  0  0]
 [ 0 -1 -1  0  0]]
effectivity: 650
